In [300]:
import cv2
import json
import geojson
import numpy as np
from geojson import Feature, FeatureCollection, Polygon, dump

In [301]:
def get_bounds():
    n_extent = 21.5439876
    e_extent = 87.118996
    s_extent = 21.5339876
    w_extent = 87.108996
    return n_extent, e_extent, s_extent, w_extent

In [302]:
def load_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGBA)
    height, width, *_ = img.shape
    for y in range(height):
        for x in range(width):
            if (img[y, x] == [0, 0, 0, 255]).all():
                img[y, x] = [0, 0, 0, 0]  
    return img

In [303]:
def trim_image(img):   
    alpha = img[..., 3]
    y_nonzero, x_nonzero = np.nonzero(alpha)
    res = alpha[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)]
    _, res = cv2.threshold(res, 64, 255, cv2.THRESH_BINARY)
    return res

In [304]:
def get_approxes(im, precision=0.0001):
    contours, *_ = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    approxes = []
    for contour in contours:
        perim = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, precision * perim, True)
        approxes.append(approx)
    return approxes

In [310]:
n_extent, e_extent, s_extent, w_extent = get_bounds()
img = load_image('../data/masks/model/model.png')
img = trim_image(img)

height, width = img.shape
height_deg, width_deg = n_extent - s_extent, e_extent - w_extent
deg_pp_height, deg_pp_width = height_deg / height, width_deg / width

features = []
approxes = get_approxes(img)
for approx in approxes:
    points = []
    for i in range(approx.shape[0] + 1):
        x, y = approx[i % approx.shape[0], 0]
        lon = w_extent + x*degppEW
        lat = n_extent - y*degppNS
        points.append((lon, lat))
    features.append(Feature(geometry=Polygon([points]), properties={"key":"value"}))


feature_col = FeatureCollection(features)
with open ('../data/masks/geojson/result.json', 'w') as f:
    dump(feature_col, f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/masks/geojson/result.json'